In [4]:
# Initialize frames folder
import os
import shutil

if not os.path.exists("frames"):
    os.makedirs("frames")
else:
  # else, remove frames from existing folder
    shutil.rmtree('frames', ignore_errors=True)

In [2]:
import pymunk
import pygame
import gymnasium as gym
import random
import sys
from pymunk.pygame_util import DrawOptions

width = 600
height = 600

class Ball:
    def __init__(self, position, space, mass, collision_type, width=50, height=10):
        self.mass = mass
        self.width = width
        self.height = height
        self.shape = pymunk.Poly.create_box(None, size=(width, height))
        self.moment = pymunk.moment_for_poly(self.mass, self.shape.get_vertices())
        self.body = pymunk.Body(self.mass, self.moment)
        self.shape.body = self.body
        leg1 = pymunk.Segment(self.body, (-20, 30), (-10, 0), 3)  # 2
        leg2 = pymunk.Segment(self.body, (20, 30), (10, 0), 3)
        self.shape.collision_type = collision_type
        self.shape.body.position = position
        space.add(self.shape, self.body, leg1, leg2)


class Ground:
    def __init__(self, space, x, y, width_ground):
        self.body = pymunk.Body(x, y, body_type=pymunk.Body.STATIC)
        self.shape = pymunk.Poly.create_box(self.body, (width_ground, 10))
        self.shape.body.position = (x + width_ground // 2, y)
        self.shape.friction = 0.7
        space.add(self.shape, self.body)

class Cube:
    def __init__(self, position, space, mass, collision_type, width=50, height=10):
        self.mass = mass
        self.width = width
        self.height = height

        self.shape = pymunk.Poly.create_box(None, size=(width, height))
        self.moment = pymunk.moment_for_poly(self.mass, self.shape.get_vertices())
        self.body = pymunk.Body(self.mass, self.moment)
        self.shape.body = self.body
        
        self.shape.collision_type = collision_type
        self.shape.body.position = position
        space.add(self.shape, self.body)

class Hurtbox:
    def hurtbox(self, body1: Cube, body2: Cube) -> bool:
        '''
        This function takes in two bodies and checks if their bounding boxes overlap 

        Inputs:
        body1 (Cube): first object
        body2 (Cube): second object
        '''

        # Get the bounding boxes of the bodies
        bb1 = body1.shape.cache_bb() 
        bb2 = body2.shape.cache_bb() 

        # Check for overlap
        return bb1.intersects(bb2)

def collide(action, arbiter, space, data):
    print("Collision detected!")
    return True

# def hitbox(entity1: Ball, entity2: Ball):
#     if entity1.shape.body.position.x <= entity2.shape.body.position.x and entity1.shape.body.position.y <= entity2.shape.body.position.y <= entity1.shape.body.position.y + entity1.height:
#         if entity1.shape.body.position.x < entity2.shape.body.position.x and \
#             entity1.shape.body.position.x + entity1.width > entity2.shape.body.position.x:
#             print("Entity1 hit by Entity2 on the LEFT")
#             return True
#         elif entity2.shape.body.position.x < entity1.shape.body.position.x and \
#               entity2.shape.body.position.x + entity2.width > entity1.shape.body.position.x:
#             print("Entity1 hit by Entity2 on the RIGHT")
#             return True
#     return False

def display_video(width: int, height: int):
    import cv2
    import numpy as np
    import skvideo.io
    from IPython.display import Video

    frame_folder = "frames"
    frames = [os.path.join(frame_folder, f) for f in sorted(os.listdir(frame_folder)) if f.endswith(".png")]

    frame_array = []
    for frame_path in frames:
        img = cv2.imread(frame_path)
        if img is None:
            raise ValueError(f"Could not read frame: {frame_path}")
        frame_array.append(img)

    video_data = np.stack(frame_array, axis=0)
    skvideo.io.vwrite('output.mp4', video_data)
    Video('output.mp4', embed=True, width=width, height=height)

def main():
    pygame.init()
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Env")
    clock = pygame.time.Clock()

    draw_options = DrawOptions(screen)

    space = pymunk.Space()
    space.gravity = 0, 200.0
    ground = Ground(space, 50, 500, 500)

    # Modify the placement of the platforms
    platform1 = Ground(space, 125, 400, 125)  # First platform
    platform2 = Ground(space, 350, 400, 125)  # Second platform with a gap

    # ball = Ball((150, 200), space, 200, 1)
    # ball2 = Ball((450, 100), space, 200, 1)

    ball = Cube((150, 100), space, 4, 1, width=50, height=50)
    ball2 = Cube((450, 100), space, 3, 2, width=50, height=50)
    hurtbox = Hurtbox()

    collision_handler = space.add_collision_handler(1, 2)
    collision_handler.begin = collide
    frame_count = 0

    # Define the y-coordinate threshold for quitting
    quit_y_threshold = 575

    def apply_force_toward_each_other(ball1, ball2, force_magnitude):
        # Calculate force directions
        direction1 = (1 if ball2.shape.body.position.x > ball1.shape.body.position.x else -1, 0)
        direction2 = (1 if ball1.shape.body.position.x > ball2.shape.body.position.x else -1, 0)

        # Apply forces
        ball1.shape.body.apply_force_at_local_point((force_magnitude * direction1[0], force_magnitude * direction1[1]), (0, 0))
        ball2.shape.body.apply_force_at_local_point((force_magnitude * direction2[0], force_magnitude * direction2[1]), (0, 0))

    for _ in range(1000):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit(0)
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                sys.exit(0)

        hurtbox.hurtbox(ball, ball2)

        # Continuously move the balls toward each other
        apply_force_toward_each_other(ball, ball2, 50)

        # Quit if any ball goes below the threshold
        if ball.shape.body.position.y > quit_y_threshold or ball2.shape.body.position.y > quit_y_threshold:
            print("A ball has fallen below the threshold. Exiting...")
            sys.exit()

        screen.fill((0, 0, 0))

        # Draw the red line
        pygame.draw.line(screen, (255, 0, 0), (0, quit_y_threshold), (width, quit_y_threshold), 2)

        space.debug_draw(draw_options)
        space.step(1/50.0)
        pygame.display.update()
        clock.tick(50)
        pygame.image.save(screen, f"frames/frame_{frame_count:04d}.png")
        frame_count += 1

if __name__ == '__main__':
    main()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-01-18 13:00:49.294 Python[23023:57228176] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
Exception ignored from cffi callback <function ext_cpCollisionBeginFunc at 0x119ac6340>:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pymunk/_callbacks.py", line 203, in ext_cpCollisionBeginFunc
    x = handler._begin(Arbiter(_arb, handler._space), handler._space, handler.data)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: collide() missing 1 required positional argument: 'data'


A ball has fallen below the threshold. Exiting...


SystemExit: 

/Users/kaden/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [2]:
import cv2
import os
import numpy as np
import skvideo.io
from IPython.display import Video

# Get the list of frames
frame_folder = "frames"
frames = [os.path.join(frame_folder, f) for f in sorted(os.listdir(frame_folder)) if f.endswith(".png")]

# Convert image files to a NumPy array
frame_array = []
for frame_path in frames:
    img = cv2.imread(frame_path)
    if img is None:
        raise ValueError(f"Could not read frame: {frame_path}")
    frame_array.append(img)

# Stack frames into a single NumPy array
video_data = np.stack(frame_array, axis=0)

# Write the video
skvideo.io.vwrite('video.mp4', video_data)

Video('video.mp4', embed=True, width=width, height=height)

NameError: name 'width' is not defined